## Notebook for reading CATIE data text files

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#root folder for all the data
root_folder = r"C:\Users\Senna\Desktop\Iigaya_lab\catie\catie_text_data\catie"

In [3]:
def process_file(file_path, text_columns_start_row=0):
    """
    Processes a text file to handle numeric columns and filter NaN values.
    
    Parameters:
    - file_path: Path to the text file.
    - text_columns_start_row: Row index from where to start processing text columns (default is 1).
    
    Returns:
    - recombined_df: A DataFrame with filtered numeric and text columns.
    """
    df = pd.read_csv(file_path, delimiter='\t', low_memory=False)
    
    # first two text columns
    df_first_two_columns = df.iloc[text_columns_start_row:, :2]  
    
    # select numeric columns (all except the first two and last two)
    df_numeric = df.iloc[text_columns_start_row:, 2:-2]  
    
    # convert non-numeric values to NaN. description row excluded and added back.
    first_row = df_numeric.iloc[0]
    rest_of_df = df_numeric.iloc[1:]
    df_numeric = rest_of_df.apply(pd.to_numeric, errors='coerce')
    df_numeric = pd.concat([pd.DataFrame([first_row]), df_numeric], ignore_index=True)

    # retain last two text columns
    df_last_two_columns = df.iloc[text_columns_start_row:, -2:]  # Last two columns
    
    # filter
    non_nan_filtered = df_numeric.dropna()
    
    # Function to check if all elements except the first one are NaN
    def should_drop(column):
        return column[1:].isna().all()

    # Identify columns to drop
    columns_to_drop = [col for col in df_numeric.columns if should_drop(df_numeric[col])]

    # Drop those columns
    non_nan_columns = df_numeric.drop(columns=columns_to_drop)
    
    # Recombine the numeric subset, first two text columns, and last two text columns
    recombined_df = pd.concat([df_first_two_columns, non_nan_columns, df_last_two_columns], axis=1)

    # Check that collection_title or promoted_subjectkey are both there
    description_row = recombined_df.iloc[0]
    desc_contains_collection_title = description_row.apply(lambda x: 'collection_title' in str(x)).any()
    desc_contains_promoted_subjectkey = description_row.apply(lambda x: 'promoted_subjectkey' in str(x)).any()
    cols_contain_collection_title = 'collection_title' in recombined_df.columns
    cols_contain_promoted_subjectkey = 'promoted_subjectkey' in recombined_df.columns

    if not desc_contains_collection_title:
        print("Collection_title is missing from the column descriptions.")
    if not desc_contains_promoted_subjectkey:
        print("Promoted_subjectkey is missing from the column descriptions.")
    if not cols_contain_collection_title:
        print("Collection_title is missing from the column names.")
    if not cols_contain_promoted_subjectkey:
        print("Promoted_subjectkey is missing from the column names.")
    if desc_contains_collection_title and desc_contains_promoted_subjectkey and cols_contain_collection_title and cols_contain_promoted_subjectkey:
        return recombined_df

def process_all_files(root_folder, file_list, text_columns_start_row=0):
    """
    Processes all files in the list and returns the processed DataFrames.
    
    Parameters:
    - root_folder: Root folder containing all text files.
    - file_list: List of file names to process.
    - text_columns_start_row: Row index from where to start processing text columns (default is 1).
    
    Returns:
    - result_dict: Dictionary with file names as keys and processed DataFrames as values.
    """
    result_dict = {}
    
    for file_name in file_list:
        file_path = os.path.join(root_folder, file_name)
        print(f"Processing {file_name}...")
        
        # Process the file
        processed_df = process_file(file_path, text_columns_start_row)
        
        # Store the result in the dictionary
        if isinstance(processed_df, pd.DataFrame):
            result_dict[file_name] = processed_df
    
    return result_dict



In [4]:
def get_text_files(directory):
    text_files = [f for f in os.listdir(directory) if f.endswith('.txt') and os.path.isfile(os.path.join(directory, f))]
    return text_files

root_folder = r"C:\Users\Senna\Desktop\Iigaya_lab\catie\catie_text_data\catie"
file_list = get_text_files(root_folder)
print(file_list)

['aesposys01.txt', 'aims01.txt', 'cata01.txt', 'cgis01.txt', 'clgry01.txt', 'dai01.txt', 'demo01.txt', 'dgsposys01.txt', 'dosecomp01.txt', 'ecg01.txt', 'endphase01.txt', 'endstudy01.txt', 'fint01.txt', 'hair01.txt', 'itaq01.txt', 'keyvars01.txt', 'lab01.txt', 'maccomp01.txt', 'macvlnce01.txt', 'med01.txt', 'meddispn01.txt', 'ndar_aggregate.txt', 'ndar_subject01.txt', 'neurobatt01.txt', 'package_info.txt', 'panss01.txt', 'qol01.txt', 'sae01.txt', 'scid_ph01.txt', 'screen01.txt', 'sf1201.txt', 'surf01.txt', 'surfq01.txt', 'timeto01.txt', 'viol01.txt', 'vitals01.txt']


In [5]:
# Process all files and store the results in a dictionary
processed_data = process_all_files(root_folder, file_list)

Processing aesposys01.txt...
Processing aims01.txt...
Processing cata01.txt...
Processing cgis01.txt...
Processing clgry01.txt...
Processing dai01.txt...
Processing demo01.txt...
Processing dgsposys01.txt...
Processing dosecomp01.txt...
Processing ecg01.txt...
Processing endphase01.txt...
Processing endstudy01.txt...
Processing fint01.txt...
Processing hair01.txt...
Processing itaq01.txt...
Processing keyvars01.txt...
Processing lab01.txt...
Processing maccomp01.txt...
Processing macvlnce01.txt...
Processing med01.txt...
Processing meddispn01.txt...
Processing ndar_aggregate.txt...
Collection_title is missing from the column descriptions.
Promoted_subjectkey is missing from the column descriptions.
Collection_title is missing from the column names.
Promoted_subjectkey is missing from the column names.
Processing ndar_subject01.txt...
Processing neurobatt01.txt...
Processing package_info.txt...
Collection_title is missing from the column descriptions.
Promoted_subjectkey is missing from

In [6]:
# Merge all dfs which have promoted_subjectkey and a collection_title
# This merges on all shared columns, doing outer to make sure all rows are kept. 
# NaNs will go in where info does not exist for that row.

# Initial df 
merged_df = list(processed_data.values())[0]

# Merge remaining dfs
for key in list(processed_data.keys())[1:]:
    merged_df = pd.merge(merged_df, processed_data[key], how='outer')

merged_df

,collection_id,dataset_id,src_subject_id,visitid,copyid,truncvis,visday,anygenae,reportid,mdsev,...,b1_wt,b1b_wt,b2_wt,b3_wt,c_wt,p_gain,bmi_cat,heart_rate,height_std,weight_std
0,collection_id,dataset_id,Subject ID how it's defined in lab/project,MetaTrial Visit ID,Copy ID for multi-copy forms,Truncated Visit Number,Number of days from study baseline to date of ...,Any general adverse events (AE),Number of AE Report by Patient,Physicians assessment of the severity of the AE,...,Phase 1/1A Baseline Weight,Phase 1B Baseline Weight,Phase 2 Baseline Weight,Phase 3 Baseline Weight,Phase-Specific CFB Weight,Phase Specific Percent Wt Gain,Body Mass Index Categorized,heart rate,Height - Standard Unit,Weight - Standard Unit
1,2081,8980,2341,400,0,0,1,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2081,8980,2341,400,0,0,1,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2081,8980,2341,400,0,0,1,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2081,8980,2341,400,0,0,1,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681109,2081,8976,1636,5800,NaN,16,487,NaN,NaN,NaN,...,152.0,NaN,165.0,165.0,0.0,0.0,NaN,70.0,NaN,165.0
681110,2081,8976,2745,200,NaN,-1,-1,NaN,NaN,NaN,...,189.0,NaN,NaN,NaN,NaN,NaN,3.0,84.0,70.0,189.0
681111,2081,8976,1165,4000,NaN,18,505,NaN,NaN,NaN,...,220.0,NaN,224.0,228.0,12.0,5.263158,NaN,82.0,NaN,240.0
681112,2081,8976,1165,200,NaN,-1,-2,NaN,NaN,NaN,...,220.0,NaN,224.0,228.0,NaN,NaN,3.0,80.0,73.0,220.0


In [7]:
#Take a look at how many rows there are for each promoted_subjectkey
merged_df['promoted_subjectkey'].value_counts()

NDAR_INVYR649KT1       1398
NDAR_INVKW099DZA       1276
NDAR_INVUK248FD1       1257
NDAR_INVUJ397VYA       1207
NDAR_INVGM160JV7       1071
                       ... 
NDAR_INVWX303CE3         66
NDAR_INVJT616KT0         63
NDAR_INVEG659ZPZ         62
NDAR_INVKE735LYG         60
promoted_subjectkey       1
Name: promoted_subjectkey, Length: 1461, dtype: int64

In [8]:
# Adding 2nd row of variable descriptions.
null_row = pd.DataFrame([np.nan] * len(merged_df.columns)).T
null_row.columns = merged_df.columns

In [9]:
#concat merged_df with the NaN's as 2nd row
merged_df= pd.concat([merged_df.iloc[:1], null_row, merged_df.iloc[1:]]).reset_index(drop=True)

In [10]:
# Mapping for the variable descriptions
variable_map = {
    'EPS': 'Extrapyramidal symptoms',
    'CGIS': 'clinical global impression scale; severity of psychopathology on a scale of 1 to 7',
    'ECG' : 'electrocardiogram',
    'QT': "ECG reading; beginning of Q wave to end of T wave",
    'SF_12': 'Medical Outcome Study Short Form-12; 12 item questionnaire measures general health status and functioning',
    'BRS':'Barnses Akathisia Scale; movement disorder severity',
    'AE' : 'Adverse events',
    'Calgary' : ' Calgary depression scare for Schizophrenia',
    'DAI' : 'drug attitude inventory',
    'ITAQ' : 'insight into treatment attitude questionnaire',
    'WRAT-3' : 'wide range achievement test-3, reading subset',
    'COWAT' : 'controlled oral word association test'
    # Add more mappings
}


def variable_description(df, variable_map):
    # Work only on the first two rows
    first_two_rows = df.iloc[:2].copy()

    # Loop through columns in the first row
    for col in first_two_rows.columns:
        first_row_value = first_two_rows.at[0, col]
        
        # Check if any of the words in variable_map are present in the first row value
        for key, value in variable_map.items():
            if pd.notna(first_row_value) and key in str(first_row_value):  

                # Populate the second row with the mapped description 
                if pd.isna(first_two_rows.at[1, col]):
                    first_two_rows.at[1, col] = value

    # Update the original DataFrame for the first two rows
    df.iloc[:2] = first_two_rows


variable_description(merged_df, variable_map)

In [11]:
# test 
print(merged_df.columns.get_loc('b1_obj'))
print(merged_df.columns.get_loc('epsmean'))

75
72


In [12]:
# test
print(merged_df.iloc[:3, 70:76])

                        sevscore  \
0  Total Movement Severity score   
1                            NaN   
2                            NaN   

                                              brsobj                  epsmean  \
0                        BRS Obj/Subject Items Score     EPS Scale Mean Score   
1  Barnses Akathisia Scale; movement disorder sev...  Extrapyramidal symptoms   
2                                                NaN                      NaN   

                                b1_score                      b1_index  \
0  Movement Severity Score Phase 1/1A BL  Severity Index Phase 1/1A BL   
1                                    NaN                           NaN   
2                                    NaN                           NaN   

                                              b1_obj  
0                      BRS Obj/Subject Phase 1/1A BL  
1  Barnses Akathisia Scale; movement disorder sev...  
2                                                NaN  
